# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
# Loading weather data csv in which "duplicate" cities have been removed (identical weather conditions and suburbs of same city)
weather_data = pd.read_csv("output_data/WeatherPy_trim.csv")
weather_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,zhigansk,66.7697,123.3711,-35.84,82,100,18.57,RU,1609967452
1,ust-nera,64.5667,143.2000,-28.23,75,50,4.79,RU,1609967389
2,talaya,61.3833,152.7500,-28.17,82,100,1.77,RU,1609967355
3,chegdomyn,51.1167,133.0167,-26.66,84,93,0.85,RU,1609967411
4,taksimo,56.3444,114.8794,-26.00,78,93,1.43,RU,1609967426


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# Configuring gmaps and setting figure size to accomodate full world map, showing all cities in dataframe
gmaps.configure(api_key=g_key)
figure_layout = {'width': '900px', 'height' : '600px'}

In [71]:
# Setting city coordinates as locations variable and humidity data as humidity variable for heatmap creation
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

# Initiating figure creation centered at 40 degrees latitude and 0 degrees longitude, slightly zoomed, and at size from above
fig = gmaps.figure(center=(40,0), zoom_level = 1.8, layout = figure_layout)

# Defining heatmap layer to display points at each city coordinate pair, weighted by that city's humidity value
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

# Adding heatmap layer to figure
fig.add_layer(heat_layer)

# Displaying figure
fig

Figure(layout=FigureLayout(height='600px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
# I would prefer a few clouds over extremely dry air
# Defining my ideal weather conditions as in the 70s (starting above 70F), wind speed less than 10mph, cloudiness less than 10%,
# and Humidity between 40-80% (non-inclusive) and storing the trimmed dataframe in the hotel_df variable
hotel_df = pd.DataFrame(weather_data.loc[((weather_data['Max Temp'] > 70) & (weather_data['Max Temp'] < 80) &
                                          (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] < 10) &
                                         (weather_data['Humidity'] > 40) & (weather_data['Humidity'] < 80)), :])
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
376,pidigan,17.5700,120.5903,71.98,79,8,3.58,PH,1609967348
377,weston,26.1004,-80.3998,72.00,51,0,4.34,US,1609967191
381,bosaso,11.2842,49.1816,72.23,66,0,2.46,SO,1609967456
405,consolacion del sur,22.5047,-83.5136,74.66,51,0,6.73,CU,1609967404
415,sal rei,16.1833,-22.9167,75.20,78,0,8.05,CV,1609967461
441,bull savanna,17.8868,-77.5902,76.66,61,0,4.81,JM,1609967455
458,zarate,-34.0981,-59.0286,78.01,61,0,5.01,AR,1609967287
480,tizimin,21.1500,-88.1500,79.20,58,0,6.76,MX,1609967333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
# Adding the Hotel Name column
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
376,pidigan,17.5700,120.5903,71.98,79,8,3.58,PH,1609967348,
377,weston,26.1004,-80.3998,72.00,51,0,4.34,US,1609967191,
381,bosaso,11.2842,49.1816,72.23,66,0,2.46,SO,1609967456,
405,consolacion del sur,22.5047,-83.5136,74.66,51,0,6.73,CU,1609967404,
415,sal rei,16.1833,-22.9167,75.20,78,0,8.05,CV,1609967461,
441,bull savanna,17.8868,-77.5902,76.66,61,0,4.81,JM,1609967455,
458,zarate,-34.0981,-59.0286,78.01,61,0,5.01,AR,1609967287,
480,tizimin,21.1500,-88.1500,79.20,58,0,6.76,MX,1609967333,


In [78]:
# Setting the parameters for hotel API search (radius= 5000m)
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# API call iterating through the rows in hotel_df (cities with ideal weather)
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    
    # Adding hotel name from json response to "Hotel Name" column of hotel_df
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        pass

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
376,pidigan,17.5700,120.5903,71.98,79,8,3.58,PH,1609967348,Strutz Art Garden Resort
377,weston,26.1004,-80.3998,72.00,51,0,4.34,US,1609967191,Bonaventure Resort & Spa
381,bosaso,11.2842,49.1816,72.23,66,0,2.46,SO,1609967456,Nayruus Hotel
415,sal rei,16.1833,-22.9167,75.20,78,0,8.05,CV,1609967461,Hotel Riu Karamboa
441,bull savanna,17.8868,-77.5902,76.66,61,0,4.81,JM,1609967455,Eves rock away villa
458,zarate,-34.0981,-59.0286,78.01,61,0,5.01,AR,1609967287,Hotel Torreblanca
480,tizimin,21.1500,-88.1500,79.20,58,0,6.76,MX,1609967333,Hotel Roazi


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
figure_layout = {'width': '900px', 'height' : '600px'}

fig = gmaps.figure(center=(40,0), zoom_level = 1.8, layout = figure_layout)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='900px'))